In [1]:
import tkinter as tk
from tkinter import messagebox
import sys
import cv2
import os
from PIL import Image
import numpy as np


In [2]:
window=tk.Tk()
window.title("Face Recognition System")

#Get User Details
l3=tk.Label(window,text="ID", font=("Arial",16))
l3.grid(column=0,row=0)
t3=tk.Entry(window,width=30,bd=5)
t3.grid(column=5,row=0)


l1=tk.Label(window,text="Name", font=("Arial",16))
l1.grid(column=0,row=4)
t1=tk.Entry(window,width=30,bd=5)
t1.grid(column=5,row=4)


l2=tk.Label(window,text="Designation", font=("Arial",16))
l2.grid(column=0,row=6)
t2=tk.Entry(window,width=30,bd=5)
t2.grid(column=5,row=6)

recognizer = cv2.face.LBPHFaceRecognizer_create()
facecascade= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eyecascade=cv2.CascadeClassifier("haarcascade_eye.xml")

#Generate Data
def button():
    if(t1.get()=='' or t2.get()=='' or t1.get().isdigit() ):
         messagebox.showinfo("Please! Enter Name and Designation")
    else:
        facecascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        video_capture = cv2.VideoCapture(0)
        id= t3.get()
        img_id=1
        name=t1.get()
        designation=t2.get()
       

        while True:
            # Capture frame-by-frame
            ret, frame = video_capture.read()
            roi_color=frame
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces =  facecascade.detectMultiScale(gray,1.2,5)
            for (x,y,w,h) in faces:
                roi_color = frame[y:y+h, x:x+w]
            img_id+=1
            face=cv2.resize(roi_color,(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            image_name=t1.get()
            file_name_path ="data/"+name+"."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)
            cv2.putText(face,name,(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow("Cropped face",face)
            if cv2.waitKey(1)==13 or int(img_id==10) :
                break
        video_capture.release() 
        cv2.destroyAllWindows()
        messagebox.showinfo("Samples are collected,Thank you!")
        faces,Ids = trainbutton('data')
        recognizer.train(faces, np.array(Ids))
        recognizer.save('trainner.yml')
       
                                                                                                  
#tarin
def trainbutton( data_dir):
   # train_classifier("/FacerecogwithGui/data")
    
    data_dir = "data";
    path=[os.path.join(data_dir,f) for f in os.listdir(data_dir)]
     #create empth face list
    faceSamples=[]
    #create empty ID list
    Ids=[]
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in path:
        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces=facecascade.detectMultiScale(imageNp)
        #If a face is there then append that in the list as well as Id of it
        for (x,y,w,h) in faces:
            faceSamples.append(imageNp[y:y+h,x:x+w])
            Ids.append(Id)
    return faceSamples,Ids

#Generate Data Button   
b1=tk.Button(window,text="Generate Data",font=("Arial",16),bg='pink',fg='black',command=button)
b1.grid(column=6,row=4)

        
 #trainandrecog
def recogbutton():
    def draw_boundary(img,classifer,eyecascade,scaleFactor,minNeighbors,color,text,clf):
        gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        features = classifer.detectMultiScale(gray_image,scaleFactor,minNeighbors)
        coords=[]
        gray_eyeimage=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        featureseye = eyecascade.detectMultiScale(gray_image,scaleFactor,minNeighbors)
       
        #fetching name
        data_dir = "data";
        path=[os.path.join(data_dir,f) for f in os.listdir(data_dir)]
        user_name=[]
        for imagePath in path:
            #loading the image and converting it to gray scale
            pilImage=Image.open(imagePath).convert('L')
            #Now we are converting the PIL image into numpy array
            imageNp=np.array(pilImage,'uint8')
            #getting the Id from the image
            user=os.path.split(imagePath)[-1].split(".")[0]
            faces=facecascade.detectMultiScale(imageNp)
            for (x,y,w,h) in faces:
                user_name.append(user)
        
        for(x,y,w,h) in featureseye:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        
        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            idn,confidence= clf.predict(gray_image[y:y+h,x:x+w])
            #confidence = int(100*(1-pred/300))

            if (confidence > 90):
                confidence = "  {0}%".format(round(100 - confidence))
                cv2.putText(img,str(user_name[idn]),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
                cv2.putText(img, str(confidence),
                            (x+5,y+h-5), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255,255,0), 1) 
            else:
                cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
                cv2.putText(img, str(0),
                            (x+5,y+h-5), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255,255,0), 1) 

            coords=[x,y,w,h]
        return  coords
    def recognize(img,clf,facecascade,eyecascade):
        coords= draw_boundary(img,facecascade,eyecascade,1.5,5,(255,255,255),"Face",clf)
        return img
   
    recognizer .read("trainner.yml") 
    if(recognizer==""):
        messagebox.showinfo("Empty Dataset")
    else:
        video_capture = cv2.VideoCapture(0)

        while True:
            ret,img = video_capture.read()
            img = recognize(img,recognizer ,facecascade,eyecascade)
            cv2.imshow("face Detection",img)
            if cv2.waitKey(1)==13:
                break

        video_capture.release()
        cv2.destroyAllWindows()
    
#Recognize Button                
b3=tk.Button(window,text="Recognize",font=("Arial",16),bg='pink',command= recogbutton)
b3.grid(column=7,row=4)      




window.geometry("600x150")
window.mainloop()